# Install

In [1]:
!pip install "transformers>=4.50.0" torch pillow

# Imports

In [2]:
import json
import re
import os
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from PIL import Image

# Model and Tokenizer

In [3]:
MODEL = "Qwen/Qwen2.5-VL-7B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL, device_map="auto"  # uses GPU if available
)
processor = AutoProcessor.from_pretrained(MODEL)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

# Helper Functions

In [5]:
def build_prompt(item, typee="text"):
    prompt = (
        f"سوال: {item['question']}\n"
        f"گزینه‌ها:\n" +
        "\n".join([f"{i+1}) {opt}" for i, opt in enumerate(item["options"])])
    )

    if typee == "text+image" and "hint" in item:
        prompt += f"\nراهنمایی: {item['hint']}"

    prompt += "\n\nفقط شماره گزینه صحیح را جواب بده."
    return prompt

def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def ask_model(prompt, image_path=None):
    messages = [{"role": "user", "content": []}]
    messages[0]["content"].append({"type": "text", "text": prompt})

    if image_path is not None:
        image = load_image(image_path, size=(224, 224))
        messages[0]["content"].append({"type": "image", "image": image})

    input_ids = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.inference_mode():
        gen_ids = model.generate(input_ids=input_ids, max_new_tokens=8)

    new_tokens = gen_ids[:, input_ids.shape[1]:]
    raw_text = processor.batch_decode(new_tokens, skip_special_tokens=True)[0].strip()
    return raw_text

def evaluate_item(item, typee):
    prompt = build_prompt(item, typee)

    image_path = None
    if typee == "image":
        image_path = os.path.join("drive/MyDrive/NLP_WH3/image_inference", f"{item['id']}.jpg")
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")

    raw = ask_model(prompt, image_path)

    print("Q", item["id"] ,": ",item["question"])
    print("Model output:", raw)

def evaluate_batch(batch, typee="text"):
    for item in batch:
        raw = evaluate_item(item, typee)


def load_image(path_or_url, size=(224, 224)):
    image = Image.open(path_or_url).convert("RGB")
    image = image.resize(size, Image.Resampling.LANCZOS)
    return image

# Text Questions

In [ ]:
with open("TextTest_merged.json", "r", encoding="utf-8") as f:
    qa_list = json.load(f)

batches = list(chunk_list(qa_list, 10))

In [ ]:
evaluate_batch(batches[0], typee="text")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q 1 :  چه ویژگی‌ای باعث شده رود ردونه در شهرستان باشت به «بهشت گمشده جنوب ایران» معروف شود؟
Model output: 2
Q 2 :  اشکفت پیچاب در شهرستان باشت چه نوع آثاری را در خود جای داده است؟
Model output: 2
Q 3 :  ستون‌های سنگی دوگور دوپا در نزدیکی روستای شوش چه کاربری احتمالی داشته‌اند؟
Model output: 2
Q 4 :  قلعه تاریخی دیشموک برای تأمین آب آشامیدنی ساکنان از چه سیستمی استفاده می‌کرده است؟
Model output: 2
Q 5 :  چه ویژگی‌ای باعث شده روستای هنجن در نطنز آب و هوای مطبوعی داشته باشد؟
Model output: 2
Q 6 :  کدام‌یک از آثار تاریخی زیر در محله افوشته نطنز قرار ندارد؟
Model output: 3
Q 7 :  نمکزار خارا در کدام فصل به دلیل تابش آفتاب و تبخیر آب، اشکال هندسی زیبایی از رسوبات نمک ایجاد می‌کند؟
Model output: 2) بهار و تابستان
Q 8 :  کدام گونه پرنده در منطقه شکار ممنوع گلاته یافت می‌شود؟
Model output: 4
Q 9 :  گنبدخانه مسجد جامع کوهپایه با الهام از چه نوع معماری ساخته شده است؟
Model output: 2
Q 10 :  حمام قدیمی کلوان در نایین به کدام دوره تاریخی مربوط است؟
Model output: 2


In [ ]:
evaluate_batch(batches[1], typee="text")

Q 11 :  قلعه قمیشلو به دستور چه کسی در دوره قاجار ساخته شد؟
Model output: 1) ناصرالدین شاه
Q 12 :  امامزاده سید نجم الدین در کدام روستا واقع شده است؟
Model output: 3
Q 13 :  آبشار شهر شیب در کدام روستای شهرستان بشاگرد قرار دارد؟
Model output: 3
Q 14 :  آرامگاه علمای کوخرد چه نامی به عنوان اثر ملی ثبت شده است؟
Model output: 1
Q 15 :  بلندترین نقطه تپه‌های ریگ بلند در آران و بیدگل چه ارتفاعی دارد؟
Model output: 3
Q 16 :  کدام گونه جانوری نادر در منطقه حفاظت‌شده بهرام گور یافت می‌شود؟
Model output: 3
Q 17 :  مجموعه کاخ سروستان توسط چه کسی در دوره ساسانی ساخته شد؟
Model output: 4
Q 18 :  بقعه شاه فخرالدین احمد در کدام شهر قرار دارد؟
Model output: 2
Q 19 :  کاخ تچر در تخت جمشید به کدام پادشاه هخامنشی اختصاص داشت؟
Model output: 3
Q 20 :  خانه تاریخی معظم‌الملک میر پنج در کدام شهر قرار دارد و به چه دوره‌ای تعلق دارد؟
Model output: 3


In [ ]:
evaluate_batch(batches[2], typee="text")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q 21 :  پل اورگان در چادگان توسط چه کسی ساخته شده و بر روی کدام رودخانه قرار دارد؟
Model output: 3
Q 22 :  روستای انالوجه در چادگان چه ویژگی بارزی در معماری و موقعیت جغرافیایی دارد؟
Model output: 1
Q 23 :  منطقه گردشگری موگرمون در لنده کهگیلویه و بویراحمد به چه دلیل شهرت دارد؟
Model output: 1) رودخانه زلال و گیاه دارویی بولاغ اوتی
Q 24 :  پل تاریخی زمان‌خان در نزدیکی شهرکرد توسط چه کسی ساخته شده و متعلق به کدام دوره است؟
Model output: 2
Q 25 :  روستای سوادجان در سامان چه ویژگی بارزی در موقعیت و بافت خود دارد؟
Model output: 1
Q 26 :  جزیره نَخیلو در خلیج فارس چه نوع جزیره‌ای است؟
Model output: 2
Q 27 :  پل پاتاوه در کدام منطقهٔ حفاظت‌شده قرار دارد؟
Model output: 3
Q 28 :  دریاچه طَشک در کنار کدام پهنهٔ آبی قرار دارد؟
Model output: 3
Q 29 :  کدام ساحل برای قایق‌سواری، اسب‌سواری و شنا مشهور است؟
Model output: 3
Q 30 :  کدام یک از امکانات در پارک جنگلی عامری فراهم است؟
Model output: 3


In [ ]:
evaluate_batch(batches[3], typee="text")

Q 31 :  بازار گناوه به چه لقبی مشهور است؟
Model output: 2
Q 32 :  جزیرهٔ میرمهنا نام خود را از کدام شخصیت تاریخی گرفته است؟
Model output: 3
Q 33 :  ایوان‌های برج خان گناوه با چند ستون به زمین متصل شده‌اند؟
Model output: 3
Q 34 :  قلعهٔ نصوری در بندر طاهری مربوط به کدام دورهٔ تاریخی است؟
Model output: 2
Q 35 :  در موزهٔ سیراف بیشتر چه نوع آثاری نگهداری می‌شود؟
Model output: 1
Q 36 :  چشمهٔ آبگرم میانلو بیشتر به خاطر کدام ویژگی گردشگری شناخته می‌شود؟
Model output: 2
Q 37 :  پل تاریخی فلاورجان روی کدام رودخانه ساخته شده است؟
Model output: 3
Q 38 :  گلستان‌کوه به رویش گستردهٔ کدام گل مشهور است؟
Model output: 2
Q 39 :  مدرسهٔ مریم‌بیگم صفوی با کدام الگوی معماری شناخته می‌شود؟
Model output: 2
Q 40 :  غار هِیکَل از نظر تیپولوژیِ غارها در کدام دسته قرار می‌گیرد؟
Model output: 3


In [ ]:
evaluate_batch(batches[4], typee="text")

Q 41 :  طراحی و بازسازی حافظیه بر پایه طرح کدام معمار انجام شد؟
Model output: 3
Q 42 :  آرامگاه سعدی در همسایگی کدام باغ قرار دارد؟
Model output: 3
Q 43 :  ارگ کریم‌خان در دورهٔ قاجار چه کاربری داشت؟
Model output: 1) دارالحکومهٔ امرای فارس
Q 44 :  مسجد نصیرالملک به چه لقبی مشهور است؟
Model output: 2
Q 45 :  کدام بنا ورودی اصلی بازار اصفهان در میدان نقش‌جهان است؟
Model output: 1
Q 46 :  طول سی‌وسه‌پل تقریباً چند متر است؟
Model output: 295
Q 47 :  گنبد نمکی جاشک در کدام استان قرار دارد؟
Model output: 3
Q 48 :  تالاب چغاخور در ارتفاعات کدام رشته‌کوه قرار دارد؟
Model output: 3
Q 49 :  کلوت‌های کوه مند عمدتاً بر اثر کدام فرایندها شکل گرفته‌اند؟
Model output: 1) فرسایش آبی و بادی
Q 50 :  کاروانسرای امین‌آباد به کدام ویژگی شاخص مشهور است؟
Model output: 2


# Text+Hint Questions

In [ ]:
with open("ImageTextTest_merged.json", "r", encoding="utf-8") as f:
    qa_list = json.load(f)

batches = list(chunk_list(qa_list, 10))

In [ ]:
evaluate_batch(batches[0], typee="text+image")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q 1 :  قلعهٔ اژدهاپیکر در کدام شهرستان ایران واقع شده است؟
Model output: 3
Q 2 :  بازار قیصریه لار عمدتاً با کدام مصالح ساخته شده است؟
Model output: 1
Q 3 :  باغ فین کاشان در کدام فهرست جهانی ثبت شده است؟
Model output: 1) میراث جهانی یونسکو
Q 4 :  محوطهٔ باستانی سیلک کاشان شامل بقایای کدام نوع سازه مشهور است؟
Model output: 2
Q 5 :  این آرامگاه سنگی باشکوه متعلق به کدام پادشاه هخامنشی است؟
Model output: 2
Q 6 :  این دهکدهٔ گردشگری ساحلی با کلبه‌های شناور در کدام شهر قرار دارد؟
Model output: 3
Q 7 :  این تصویر مربوط به کدام منطقهٔ حفاظت‌شدهٔ ساحلی است؟
Model output: 3
Q 8 :  اسکلهٔ نخل تقی در کدام شهر واقع شده است؟
Model output: 3
Q 9 :  این آبشار در کدام روستا از بخش عسلویه قرار دارد؟
Model output: 3
Q 10 :  جنگل‌های حرا در پارک ملی نایبند به کدام ویژگی طبیعی مشهورند؟
Model output: 2


In [ ]:
evaluate_batch(batches[1], typee="text+image")

Q 11 :  این قلعه تاریخی در کدام شهرستان ایران واقع شده است؟
Model output: 4
Q 12 :  این جزیرهٔ حفاظت‌شده در کدام استان ایران قرار دارد؟
Model output: 2
Q 13 :  این پل تاریخی بر روی کدام رودخانه ساخته شده است؟
Model output: 3
Q 14 :  این مجموعهٔ تفریحی-طبیعی در کدام شهر قرار دارد؟
Model output: 3
Q 15 :  این بنای چهارایوانی تاریخی در کدام شهر قرار دارد؟
Model output: 3
Q 16 :  این پدیدهٔ طبیعی شبیه «آبشارهای نمکی» در کدام استان ایران قرار دارد؟
Model output: 3
Q 17 :  این کاروانسرای تاریخی به کدام ویژگی شاخص معماری مشهور است؟
Model output: 2
Q 18 :  این پل تاریخی در اصل در کدام دورهٔ ایران باستان بنا شده است؟
Model output: 1
Q 19 :  سنگ‌نگاره‌های این تنگهٔ کوهستانی به کدام دوره تاریخی نسبت داده می‌شوند؟
Model output: 3
Q 20 :  در این مجموعهٔ باغ-عمارت، کدام مورد نیز برای بازدید عموم در دسترس است؟
Model output: 3


In [ ]:
evaluate_batch(batches[2], typee="text+image")

Q 21 :  این قلعه تاریخی در کدام روستا از شهرستان بندر دیلم قرار دارد؟
Model output: 3
Q 22 :  این منطقه ساحلی پوشیده از درختان کهور و اکالیپتوس چه نام دارد؟
Model output: 3
Q 23 :  این پارک جنگلی در شهرستان دیلم چه نام دارد؟
Model output: 3
Q 24 :  این تصویر مربوط به کدام تالاب استان بوشهر است؟
Model output: 2
Q 25 :  این جزیره غیرمسکونی در خلیج فارس چه نام دارد؟
Model output: 3
Q 26 :  این گنبد نمکی مشهور در کدام استان قرار دارد؟
Model output: 3
Q 27 :  این ساحل شنی با آب‌های شفاف به کدام شهرستان تعلق دارد؟
Model output: 3
Q 28 :  این جنگل حرا در کدام بخش از خلیج فارس قرار دارد؟
Model output: 2
Q 29 :  این تپه‌های شنی بلند در کدام استان ایران قرار دارند؟
Model output: 3
Q 30 :  این تالاب فصلی در نزدیکی کدام شهر قرار دارد؟
Model output: 3


# Text+Image Questions

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
with open("ImageTextTest_merged.json", "r", encoding="utf-8") as f:
    qa_list = json.load(f)

batches = list(chunk_list(qa_list, 10))

In [9]:
evaluate_batch(batches[0], typee="image")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q 1 :  قلعهٔ اژدهاپیکر در کدام شهرستان ایران واقع شده است؟
Model output: 3
Q 2 :  بازار قیصریه لار عمدتاً با کدام مصالح ساخته شده است؟
Model output: 1
Q 3 :  باغ فین کاشان در کدام فهرست جهانی ثبت شده است؟
Model output: 1
Q 4 :  محوطهٔ باستانی سیلک کاشان شامل بقایای کدام نوع سازه مشهور است؟
Model output: 2
Q 5 :  این آرامگاه سنگی باشکوه متعلق به کدام پادشاه هخامنشی است؟
Model output: 2
Q 6 :  این دهکدهٔ گردشگری ساحلی با کلبه‌های شناور در کدام شهر قرار دارد؟
Model output: 1
Q 7 :  این تصویر مربوط به کدام منطقهٔ حفاظت‌شدهٔ ساحلی است؟
Model output: 3
Q 8 :  اسکلهٔ نخل تقی در کدام شهر واقع شده است؟
Model output: 2
Q 9 :  این آبشار در کدام روستا از بخش عسلویه قرار دارد؟
Model output: 3
Q 10 :  جنگل‌های حرا در پارک ملی نایبند به کدام ویژگی طبیعی مشهورند؟
Model output: 2


In [ ]:
evaluate_batch(batches[1], typee="image")

Q 11 :  این قلعه تاریخی در کدام شهرستان ایران واقع شده است؟
Model output: 4
Q 12 :  این جزیرهٔ حفاظت‌شده در کدام استان ایران قرار دارد؟
Model output: 2
Q 13 :  این پل تاریخی بر روی کدام رودخانه ساخته شده است؟
Model output: 3
Q 14 :  این مجموعهٔ تفریحی-طبیعی در کدام شهر قرار دارد؟
Model output: 3
Q 15 :  این بنای چهارایوانی تاریخی در کدام شهر قرار دارد؟
Model output: 3
Q 16 :  این پدیدهٔ طبیعی شبیه «آبشارهای نمکی» در کدام استان ایران قرار دارد؟
Model output: 4
Q 17 :  این کاروانسرای تاریخی به کدام ویژگی شاخص معماری مشهور است؟
Model output: جواب صحیح این س
Q 18 :  این پل تاریخی در اصل در کدام دورهٔ ایران باستان بنا شده است؟
Model output: 4
Q 19 :  سنگ‌نگاره‌های این تنگهٔ کوهستانی به کدام دوره تاریخی نسبت داده می‌شوند؟
Model output: 3
Q 20 :  در این مجموعهٔ باغ-عمارت، کدام مورد نیز برای بازدید عموم در دسترس است؟
Model output: جواب صحیح: 4


In [10]:
evaluate_batch(batches[2], typee="image")

Q 21 :  این قلعه تاریخی در کدام روستا از شهرستان بندر دیلم قرار دارد؟
Model output: 3
Q 22 :  این منطقه ساحلی پوشیده از درختان کهور و اکالیپتوس چه نام دارد؟
Model output: 3
Q 23 :  این پارک جنگلی در شهرستان دیلم چه نام دارد؟
Model output: 2
Q 24 :  این تصویر مربوط به کدام تالاب استان بوشهر است؟
Model output: 2
Q 25 :  این جزیره غیرمسکونی در خلیج فارس چه نام دارد؟
Model output: 2
Q 26 :  این گنبد نمکی مشهور در کدام استان قرار دارد؟
Model output: 3
Q 27 :  این ساحل شنی با آب‌های شفاف به کدام شهرستان تعلق دارد؟
Model output: 2
Q 28 :  این جنگل حرا در کدام بخش از خلیج فارس قرار دارد؟
Model output: 2
Q 29 :  این تپه‌های شنی بلند در کدام استان ایران قرار دارند؟
Model output: 3
Q 30 :  این تالاب فصلی در نزدیکی کدام شهر قرار دارد؟
Model output: 2
